# pCO$_{2}$ Taylor Expansion
Conducting a Taylor Expansion of the impacts of DIC, Alk, SST, and SALT on pCO$_{2}$ during a climate variability event in EBUS. This is inspired from Nikki Lovenduski's 2007 and 2015 papers.

----

### Expansion

$ \Delta pCO_{2} = \frac{\partial pCO_{2}}{\partial DIC}\Delta DIC + \frac{\partial pCO_{2}}{\partial ALK}\Delta ALK + \frac{\partial pCO_{2}}{\partial SST}\Delta SST + \frac{\partial pCO_{2}}{\partial SALT}\Delta SALT $

### Sensitivity Terms

$ 	\frac{\partial pCO_{2}}{\partial SST} \approx 0.0423^{o}C^{-1}\cdot pCO_{2} $

$ 	\frac{\partial pCO_{2}}{\partial SALT} \approx \frac{pCO_{2}}{SALT} $

$ 	\frac{\partial pCO_{2}}{\partial DIC} = \frac{pCO_{2}}{DIC}\gamma_{DIC} $


$ 	\frac{\partial pCO_{2}}{\partial ALK} = \frac{pCO_{2}}{ALK}\gamma_{ALK} $


### Buffer Factors
$ \gamma_{DIC} = \frac{3\cdot ALK\cdot DIC - 2DIC^{2}}{(2DIC - ALK)(ALK - DIC)} $

$ \gamma_{ALK} = - \frac{ALK^{2}}{(2DIC - ALK)(ALK-DIC)} $


In [1]:
import glob
import numpy as np
from numpy import mean
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
%matplotlib inline

# Functions

In [2]:
def load_forced_signal(VAR, EBU):
    fileDir = '/glade/p/work/rbrady/EBUS_BGC_Variability/' + VAR + '/' + EBU + '/filtered_output/'
    fileName = EBU.lower() + '-' + VAR + '-forced-signal-AW-chavez-800km.nc'
    ds = xr.open_dataset(fileDir + fileName)
    data = ds[VAR + '_AW'].values
    return data

# Determine Sensitivity Terms

In [ ]:
EBUs = ['CalCS', 'HumCS', 'CanCS', 'BenCS']
buffers = dict()
sensitivities = dict()
for idx in EBUs:
    pCO2 = load_forced_signal('pCO2SURF', idx)
    DIC = load_forced_signal('DIC', idx)
    SALT = load_forced_signal('SALT', idx)
    ALK = load_forced_signal('ALK', idx)
    SST = load_forced_signal('SST', idx)
    SALT = load_forced_signal('SALT', idx)
    # Compute buffer factors for DIC/ALK.
    buffer_factor = dict()
    buffer_factor['ALK'] = -mean(ALK)**2 / ((2 * mean(DIC) - mean(ALK)) * (mean(ALK) - mean(DIC)))
    buffer_factor['DIC'] = (3*mean(ALK)*mean(DIC) - 2*mean(DIC)**2) / \
                           ((2 * mean(DIC) - mean(ALK)) * (mean(ALK) - mean(DIC)))
    # Compute sensitivities
    sensitivity = dict()
    sensitivity['SST'] = 0.0423 * mean(pCO2)
    sensitivity['SALT'] = mean(pCO2)/mean(SALT)
    sensitivity['ALK'] = (mean(pCO2)/mean(ALK))*buffer_factor['ALK']
    sensitivity['DIC'] = (mean(pCO2)/mean(DIC))*buffer_factor['DIC']
    # Place in final arrays.
    buffers[idx] = buffer_factor
    sensitivities[idx] = sensitivity

In [ ]:
# Make them Pandas!
buffers = pd.DataFrame(data=buffers)
sensitivities = pd.DataFrame(data=sensitivities)

In [ ]:
# Save for later!
directory = '/glade/u/home/rbrady/projects/EBUS_BGC_Variability/data/processed/all-systems/'
buffers.to_csv(directory + 'pCO2_buffers_all_systems')
sensitivities.to_csv(directory + 'pCO2_sensitivities_all_systems')

# Compute Linear Expansion

In [3]:
def load_correlations(VAR, EBU):
    fileDir = '/glade/u/home/rbrady/projects/EBUS_BGC_Variability/data/processed/' + \
                EBU.lower() + '/'
    fileName = 'unsmoothed_' + VAR + '_vs_enso_' + EBU
    df = pd.DataFrame.from_csv(fileDir + fileName)
    return df

In [6]:
EBU = 'CalCS'
# Load in sensitivity/buffer results.
fileDir = '/glade/u/home/rbrady/projects/EBUS_BGC_Variability/data/processed/all-systems/'
buffers = pd.DataFrame.from_csv(fileDir + 'pCO2_buffers_all_systems')
buffers = buffers[EBU]
sensitivities = pd.DataFrame.from_csv(fileDir + 'pCO2_sensitivities_all_systems')
sensitivities = sensitivities[EBU]

In [11]:
# Load in correlation results
pCO2_corr = load_correlations('pCO2SURF', EBU)
DIC_corr = load_correlations('DIC', EBU)
ALK_corr = load_correlations('ALK', EBU)
SST_corr = load_correlations('SST', EBU)
SALT_corr = load_correlations('SALT', EBU)

In [10]:
pCO2_corr

,Slope,R Value,R Squared,P-Value
0,0.349349,0.098202,0.009644,8.452683e-04
1,0.469228,0.101813,0.010366,5.383722e-04
2,0.190154,0.043574,0.001899,1.393993e-01
3,0.471008,0.104148,0.010847,3.990898e-04
4,-0.189613,-0.047393,0.002246,1.078950e-01
5,0.491717,0.112263,0.012603,1.343989e-04
6,0.179506,0.037475,0.001404,2.037206e-01
7,0.417391,0.110054,0.012112,1.820780e-04
8,0.593217,0.126760,0.016068,1.595142e-05
9,0.154140,0.032608,0.001063,2.687989e-01
